In [1]:
import numpy as np
import time

from pdag.data_generator import generate_random_partialB, generate_random_MVN_data
from pdag.pdag import PDAG

ImportError: cannot import name 'PDAG' from partially initialized module 'pdag' (most likely due to a circular import) (/Users/syed_rahman2/Documents/pyPDAG/pdag/__init__.py)

In [ ]:
p = 20
n = 10*p
omega, B, A, G = generate_random_partialB(int(p/4), 
                                          int(2*p/4), 
                                          int(3*p/4),
                                          p = p,
                                          a = 0.3,
                                          b = 0.7,
                                          diag_a = 2,
                                          diag_b = 5,
                                          plot=False)
B = np.array(B)
X = generate_random_MVN_data(n, omega)

In [ ]:
pdag4 = PDAG(4)
start = time.time()
Bhat = pdag4.fit(X, 0.001, int(p/4), int(2*p/4), int(3*p/4))
#partial4.parallel_diagnostics(level=4)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

In [ ]:
pdag4.plotGraph()